In [ ]:
# default_exp core
%reload_ext autoreload
%autoreload 2

# FlyVec
> Functions that use a pretrained FlyVec model to create sparse binary representations

In [ ]:
#export
import yaml
import os
import numpy as np
from flyvec.tokenizer import GensimTokenizer
from pathlib import Path
from cached_property import cached_property
from functools import cached_property, lru_cache
from typing import *
from fastcore.test import *

## Helpers

In [ ]:
#export
def softmax(x: np.array, beta=1.0):
    """Take the softmax of 1-D vector `x` according to inverse temperature `beta`. Returns a vector of the same length as x"""
    v = np.exp(beta*x)
    return v / np.sum(v)

In [ ]:
#export
def normalize_synapses(syn: np.array, prec=1.0e-32, p=2):
    """Normalize the synapses

    Args:
        syn: The matrix of learned synapses
        prec: Noise to prevent division by 0
        p: Of the p-norm

    Returns:
        Normalized array of the given synapses
    """
    K, N = syn.shape
    nc = np.power(np.sum(syn**p,axis=1),1/p).reshape(K,1)
    return syn / np.tile(nc + prec, (1, N))

In [ ]:
#export
class FlyVec:
    """A class wrapper around a tokenizer, stop words, and synapse weights for hashing words"""

    def __init__(self, synapse_file: Union[Path, str], tokenizer_file: Union[Path, str], stopword_file: Optional[Union[Path, str]]=None, phrases_file: Optional[Union[Path, str]]=None, normalize_synapses: bool=True):

        self.synapse_file = str(synapse_file)
        self.tokenizer_file = str(tokenizer_file)
        self.stopword_file = str(stopword_file) if stopword_file is not None else None
        self.phrases_file = str(phrases_file) if phrases_file is not None else None
        self.normalize_synapses = normalize_synapses

    @classmethod
    def from_config(cls, fname: Union[Path, str]):
        """Create an instance of this class from the configuration present in the `fname` yaml file"""
        fpath = Path(fname)
        ref_dir = fpath.parent

        with open(fname, "r") as fp:
            conf = yaml.load(fp, Loader=yaml.FullLoader)

        synapse_file = ref_dir / conf["synapses"]
        tokenizer_file = ref_dir / conf["tokenizer"]
        phrases_file = ref_dir / conf["phrases"] if "phrases" in conf.keys() else None
        stopword_file = ref_dir / conf["stop_words"] if "stop_words" in conf.keys() else None
        normalize_synapses = conf.get("normalize_synapses", False)

        return cls(synapse_file, tokenizer_file, stopword_file=stopword_file, phrases_file=phrases_file, normalize_synapses=normalize_synapses)

    @cached_property
    def n_neurons(self): return self.synapses.shape[0]

    @cached_property
    def synapses(self):
        """The primary weights learned by the model"""
        print("Loading synapses...")
        syn = np.load(self.synapse_file)

        if self.normalize_synapses: return normalize_synapses(syn)
        return syn

    @cached_property
    def tokenizer(self):
        print("Loading Tokenizer...")
        return GensimTokenizer.from_file(self.tokenizer_file, self.phrases_file)

    @cached_property
    def stop_words(self):
        """Words the model should not respond to"""
        print("Loading stop words...")
        return set(np.load(self.stopword_file))

    @cached_property
    def vocab(self):
        return set(self.tokenizer.vocab)
    
    @cached_property
    def n_vocab(self): return self.tokenizer.n_vocab()
    
    @cached_property
    def unknown_embedding_info(self):
        return {
            "token": "<UNK>",
            "tok_id": 0,
            "embedding": np.zeros(self.n_neurons).astype(np.uint8)
        }
    
    def tokenize(self, sentence:str):
        return self.tokenizer.tokenize(sentence)
    
    def is_unknown_token(self, token:str):
        """Check if a token is unknown (return false) or bad (raise ValueError)"""
        if len(token) == 0:
            raise ValueError("Token cannot be the empty string")
            
        tok = self.tokenizer.tokenize(token)[0]
        tok_id = self.tokenizer.token2id(tok)
        return tok_id == 0
    
    def get_sparse_embedding(self, word: str, hash_length: int=32):
        """Get a context-independent word embedding for a given word. 
        If, when tokenized, the word is composed of multiple tokens, return the embedding of the first.
        
        Args:
            token: A token (in the vocabulary) to get the word-embedding of
            hash_length: The number of non-zero entries in the sparse embedding
            normalize_first: If true, preprocess the token to be lowercase, no punctuation, etc. 
            
        Returns:
            `np.ndarray` of shape (self.n_neurons,) and dtype np.int8
        """
        if self.is_unknown_token(word): return self.unknown_embedding_info
        
        dense_info = self.get_dense_embedding(word)
        act = dense_info['embedding']
        i_sorted = np.argsort(-act)
        act_sort = act[i_sorted]
        thr = (act_sort[hash_length - 1] + act_sort[hash_length]) / 2.0
        binary = (act > thr).astype(np.int8)
        return {
            "token": dense_info['token'],
            "id": dense_info['id'],
            "embedding": binary
        }
    
    def get_dense_embedding(self, word: str):
        """Get a context-independent word embedding for a given token
        
        Args:
            token: A token (in the vocabulary) to get the word-embedding of
            hash_length: The number of non-zero entries in the sparse embedding
            
        Returns:
            `np.ndarray` of shape (self.n_neurons,) and dtype np.float64
        """
        if self.is_unknown_token(word): return self.unknown_embedding_info
        
        token = self.tokenizer.tokenize(word)[0]
        tok_id = self.tokenizer.token2id(token)
        activation_scores = self.synapses[:, self.n_vocab + tok_id] # Target word embedding is stored in second compartment of matrix

        return {
            "token": token,
            "id": tok_id,
            "embedding": activation_scores
        }

If we are testing multiple configurations of `FlyVec` from different `yaml` files, it can be useful to cache the created objects

In [ ]:
model = FlyVec.from_config("./model_config.yaml")
hsh = model.get_sparse_embedding("hello", 32); hsh

Loading Tokenizer...
No phraser specified. Proceeding without phrases
Loading synapses...


{'token': 'hello',
 'id': 5483,
 'embedding': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

`hsh['embedding']` is non-zero for the top `hash_length` most activated neurons in our model

If you provide multiple words in the input string, `FlyVec` will provided the word vector for the first word

In [ ]:
hsh2 = model.get_sparse_embedding("hello world", 32);
assert np.all(hsh2['embedding'] == hsh['embedding'])

In [ ]:
_f = lambda x: model.get_sparse_embedding(x)
test_eq(_f("hello")['token'], "hello")
assert np.all(_f("BOXNAFS")['embedding'] == 0), "Expected unknown embedding to be all zero"
test_eq(_f("HELLO")['embedding'], _f("hello")['embedding'])
test_eq(_f("not a single token")['embedding'], _f("not")['embedding'])
test_fail(lambda: _f(""), contains="empty string")
test_eq(_f("NotARealWord")['embedding'], _f("<UNK>")['embedding'])

In [ ]:
#export
@lru_cache
def initialize_flyvec(fname):
    return FlyVec.from_config(fname)

# Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()